In [ ]:
!git clone https://github.com/BaseballCV/BaseballCV.git

In [ ]:
!pip install baseballcv

In [ ]:
!wget https://data.balldatalab.com/index.php/s/pLy7sZqqMdx3jj7/download/baseball_rubber_home_glove.zip

--2025-08-22 15:19:58--  https://data.balldatalab.com/index.php/s/pLy7sZqqMdx3jj7/download/baseball_rubber_home_glove.zip
Resolving data.balldatalab.com (data.balldatalab.com)... 69.48.207.209
Connecting to data.balldatalab.com (data.balldatalab.com)|69.48.207.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427814730 (1.3G) [application/zip]
Saving to: ‘baseball_rubber_home_glove.zip’

baseball_rubber_hom 100%[===================>]   1.33G  51.7MB/s    in 27s     

2025-08-22 15:20:26 (50.7 MB/s) - ‘baseball_rubber_home_glove.zip’ saved [1427814730/1427814730]



In [ ]:
!unzip /content/baseball_rubber_home_glove.zip

During my first attempt at training, I got messages that some images were being ignored because there labels had an extra label. So that data needs to be cleaned.

In [ ]:
import os

label_dirs = [
    '/content/baseball_rubber_home_glove/train/labels',
    '/content/baseball_rubber_home_glove/valid/labels',
    '/content/baseball_rubber_home_glove/test/labels'
]

removed_count = 0
for label_dir in label_dirs:
    for fname in os.listdir(label_dir):
        if fname.endswith('.txt'):
            path = os.path.join(label_dir, fname)
            with open(path, 'r') as f:
                lines = f.readlines()
            new_lines = [line for line in lines if not line.strip().startswith('4 ')]
            removed = len(lines) - len(new_lines)
            if removed > 0:
                with open(path, 'w') as f:
                    f.writelines(new_lines)
                removed_count += removed
                print(f"Removed {removed} class 4 label(s) from: {path}")
print(f"Total class 4 labels removed: {removed_count}")

Total class 4 labels removed: 0


Now that the data has been cleaned. Let's do a first pass training.
imgsz of 640 was used because a quick search showed it was a commonly used size that balanced both training speed and detection accuracy. https://github.com/ultralytics/yolov5/issues/5851

20 epochs was chosen just as a quick first pass to ensure training was taking place.

In [ ]:
# Clear memory before training
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from ultralytics import YOLO
model = YOLO('glove_tracking_v4_YOLOv11.pt')


# Load pretrained model
model = YOLO('glove_tracking_v4_YOLOv11.pt')

# Train (fine-tune) on dataset
model.train(data='glove_data.yaml', epochs=20, imgsz=640, batch=8)

Try a learning rate of 0.005 for fine-tuning.

In [ ]:
from ultralytics import YOLO

# Clear memory before training
import torch, gc
gc.collect()
torch.cuda.empty_cache()

# Load pretrained model
model = YOLO('glove_tracking_v4_YOLOv11.pt')

# Train (fine-tune) on dataset
model.train(data='glove_data.yaml', epochs=20, imgsz=512, batch=8, lr0=0.005)

from google.colab import files
import shutil
shutil.make_archive("/content/runs/detect/train20", 'zip', "/content/runs/detect/train20")
files.download("/content/runs/detect/train20.zip")


Keep the learning rate and 0.005 and increase the warm-up epochs from 2 to 4.

In [ ]:
from ultralytics import YOLO

# Clear memory before training
import torch, gc
gc.collect()
torch.cuda.empty_cache()



# Load pretrained model
model = YOLO('glove_tracking_v4_YOLOv11.pt')

# Train (fine-tune) on dataset
model.train(data='glove_data.yaml', epochs=20, imgsz=512, batch=8, lr0=0.005, warmup_epochs=5)

from google.colab import files
import shutil
shutil.make_archive("/content/runs/detect/train21", 'zip', "/content/runs/detect/train21")
files.download("/content/runs/detect/train21.zip")